<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/06wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [1]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-yx6DQsLACqw8pWm0udv8Jm&si=in1eMD0-wU49y7mS >}}

# 2. Imports

In [4]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [5]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. 다항분류

## A. 이항분류와 `BCEWithLogitsLoss`

`-` 데이터

In [6]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0_train = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==0])
X1_train = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==1])
X = torch.concat([X0_train,X1_train],axis=0).reshape(-1,784)
y = torch.tensor([0.0]*len(X0_train) + [1.0]*len(X1_train)).reshape(-1,1)

`-` 예전코드는 아래와 같음 (sig는 수동처리함) 

In [5]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
)
loss_fn = torch.nn.BCELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,11):
    # step1 
    netout = net(X) # netout = logits 
    yhat = torch.exp(netout) / (1 + torch.exp(netout)) # yhat = prob 
    # step2
    loss = loss_fn(yhat,y) 
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()
    #---에폭끝나고 확인할 것들---#
    acc = ((net(X).data > 0)  == y).float().mean()
    print(f"epoch = {epoc}\t acc = {acc:.4f}")

epoch = 1	 acc = 0.4677
epoch = 2	 acc = 0.4677
epoch = 3	 acc = 0.4745
epoch = 4	 acc = 0.5572
epoch = 5	 acc = 0.6995
epoch = 6	 acc = 0.8175
epoch = 7	 acc = 0.8904
epoch = 8	 acc = 0.9255
epoch = 9	 acc = 0.9444
epoch = 10	 acc = 0.9577


`#` netout(= logits) 의 특징

- $netout > 0 \Leftrightarrow sig(netout) >0.5$
- $netout < 0 \Leftrightarrow sig(netout) <0.5$

`-` 그런데 위의 코드는 아래의 코드와 같음 

In [7]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1)
)
loss_fn = torch.nn.BCEWithLogitsLoss() # <--- 여기를 바꾸고 
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,11):
    # step1 
    netout = net(X) # netout = logits 
    # yhat = torch.exp(netout) / (1 + torch.exp(netout))  # yhat = prob  # <-- 여기는주석처리
    # step2
    loss = loss_fn(netout,y) 
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()
    #---에폭끝나고 확인할 것들---#
    acc = ((net(X).data > 0)  == y).float().mean()
    print(f"epoch = {epoc}\t acc = {acc:.4f}")

epoch = 1	 acc = 0.4677
epoch = 2	 acc = 0.4677
epoch = 3	 acc = 0.4745
epoch = 4	 acc = 0.5572
epoch = 5	 acc = 0.6995
epoch = 6	 acc = 0.8175
epoch = 7	 acc = 0.8904
epoch = 8	 acc = 0.9255
epoch = 9	 acc = 0.9444
epoch = 10	 acc = 0.9577


## B. 범주형자료의 변환 

`-` 범주형자료를 숫자로 어떻게 바꿀까? 

- 실패 / 성공 $\to$ 0 / 1
- 숫자0그림 / 숫자1그림 $\to$ 0 / 1 
- 강아지그림 / 고양이그림 $\to$ 0 / 1
- 강아지그림 / 고양이그림 / 토끼그림 $\to$ 0 / 1 / 2 ?????

`-` 주입식교육: 강아지그림/고양이그림/토끼그림일 경우 숫자화시키는 방법 

- 잘못된방식: 강아지그림 = 0, 고양이그림 = 1, 토끼그림 = 2
- 올바른방식: 강아지그림 = [1,0,0], 고양이그림 = [0,1,0], 토끼그림 = [0,0,1]  ### <-- 이런방식을 원핫인코딩이라함

`-` 왜? 

- 설명1: 강아지그림, 고양이그림,  토끼그림은 서열측도가 아니라 명목척도임. 그래서 범주를 0,1,2 로 숫자화하면 평균등의 의미가 없음 (사회조사분석사 2급 스타일) 
- 설명2: 범주형은 원핫인코딩으로 해야함 ("30일만에 끝내는 실전머신러닝" 이런 책에 나오는 스타일) 
- 설명3: 동전을 한번 던져서 나오는 결과는 $n=1$인 이항분포를 따름. 주사위 한번 던져서 나오는 눈금의 숫자는 $n=1$인 다항분포를 따름. $n=1$인 이항분포의 실현값은 0,1 이고, $n=1$인 다항분포의 실현값은 [1,0,0], [0,1,0], [0,0,1] 이므로 당연히 $y_i$ 는 [1,0,0], [0,1,0], [0,0,1] 중 하나의 형태를 가진다고 가정하는게 바람직함 (이 설명이 이 중에서 가장 정확한 설명임)

## C. 실습: 3개의 클래스를 구분¶

`-` 데이터준비 

In [8]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==0])
X1 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==1])
X2 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==2])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)
y = torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2)).reshape(-1,1).float()

In [9]:
y = torch.nn.functional.one_hot(y.flatten().long()).float()

`-` 적합 

In [10]:
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,3),
)
loss_fn = torch.nn.CrossEntropyLoss() # 이름이 좀 그래.. 나같으면 CEWithLogitsLoss 라고 했을듯 
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(100):
    ## step1 
    netout = net(X)
    ## step2 
    loss = loss_fn(netout,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

In [11]:
(net(X).argmax(axis=1)  == y.argmax(axis=1)).float().mean()

tensor(0.9827)

## D. 결론 -- 외우세여

`-` 파이토치버전 // 코딩용

|분류 | netout의 의미 | 손실함수 | 
|:--:|:--:|:--:|
|이항분류 | prob | `BCELoss` | 
|이항분류 | logit | `BCEWithLogitsLoss` | 
|다항분류| probs |  NA |
|다항분류| logits |  `CrossEntropyLoss` | 

> `CrossEntropyLoss` 이거 이름이 완전 마음에 안들어요.. `CEWithLogitsLoss` 라고 하는게 더 좋을 것 같습니다. 

`-` 일반적개념 // 이론용

|분류 |오차항의가정|  마지막활성화함수 | 손실함수 | 
|:--:|:--:|:--:|:--:|
|이항분류 | 이항분포 | sigmoid^[prob=sig(logit)] |Binary Cross Entropy  | 
|다항분류| 다항분포 | softmax^[probs=soft(logits)] |  Cross Entropy |

# 4. FashionMNIST

## A. 데이터 

In [255]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset])
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float()
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset])
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float()

In [256]:
ds_train = torch.utils.data.TensorDataset(X,y)
dl_train = torch.utils.data.DataLoader(ds_train,batch_size=256,shuffle=True)
ds_test = torch.utils.data.TensorDataset(XX,yy)
dl_test = torch.utils.data.DataLoader(ds_test,batch_size=256)

## B. 간단한 신경망

In [257]:
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,10)
).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,31):
    #---에폭시작---#
    net.train()
    for Xm,ym in dl_train:
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        # Step1
        netout = net(Xm)
        # Step2
        loss = loss_fn(netout,ym) 
        # Step3
        loss.backward()
        # Step4 
        optimizr.step()
        optimizr.zero_grad()
    #---에폭끝---#
    if epoc % 5 ==0:
        net.eval()
        s = 0 
        for Xm,ym in dl_train:
            Xm = Xm.to("cuda:0")
            ym = ym.to("cuda:0")        
            s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
        acc = s / len(X)
        print(f"# of epochs = {epoc}\t train_acc = {acc:.4f}")

# of epochs = 5	 train_acc = 0.8588
# of epochs = 10	 train_acc = 0.8659
# of epochs = 15	 train_acc = 0.8779
# of epochs = 20	 train_acc = 0.8830
# of epochs = 25	 train_acc = 0.8857
# of epochs = 30	 train_acc = 0.8875


In [260]:
net.eval()
s = 0 
for Xm,ym in dl_test:
    Xm = Xm.to("cuda:0")
    ym = ym.to("cuda:0")        
    s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
acc = s / len(XX)
print(f"test_acc = {acc:.4f}")

test_acc = 0.8639


In [261]:
sum([torch.tensor(para.shape).prod().item() for para in net.parameters()])

25450

## C. 복잡한 신경망

In [262]:
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,256),
    torch.nn.ReLU(),
    torch.nn.Linear(256,10)
).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,31):
    #---에폭시작---#
    net.train()
    for Xm,ym in dl_train:
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        # Step1
        netout = net(Xm)
        # Step2
        loss = loss_fn(netout,ym) 
        # Step3
        loss.backward()
        # Step4 
        optimizr.step()
        optimizr.zero_grad()
    #---에폭끝---#
    if epoc % 5 ==0:
        net.eval()
        s = 0 
        for Xm,ym in dl_train:
            Xm = Xm.to("cuda:0")
            ym = ym.to("cuda:0")        
            s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
        acc = s / len(X)
        print(f"# of epochs = {epoc}\t train_acc = {acc:.4f}")

# of epochs = 5	 train_acc = 0.8831
# of epochs = 10	 train_acc = 0.9028
# of epochs = 15	 train_acc = 0.9183
# of epochs = 20	 train_acc = 0.9281
# of epochs = 25	 train_acc = 0.9331
# of epochs = 30	 train_acc = 0.9332


In [263]:
net.eval()
s = 0 
for Xm,ym in dl_test:
    Xm = Xm.to("cuda:0")
    ym = ym.to("cuda:0")        
    s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
acc = s / len(XX)
print(f"test_acc = {acc:.4f}")

test_acc = 0.8833


In [264]:
sum([torch.tensor(para.shape).prod().item() for para in net.parameters()])

203530

## D. 발악해보세요

`-` 아주 넓게.. 

In [265]:
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,4096),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(4096,10),
).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,31):
    #---에폭시작---#
    net.train()
    for Xm,ym in dl_train:
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        # Step1
        netout = net(Xm)
        # Step2
        loss = loss_fn(netout,ym) 
        # Step3
        loss.backward()
        # Step4 
        optimizr.step()
        optimizr.zero_grad()
    #---에폭끝---#
    if epoc % 5 ==0:
        net.eval()
        s = 0 
        for Xm,ym in dl_train:
            Xm = Xm.to("cuda:0")
            ym = ym.to("cuda:0")        
            s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
        acc = s / len(X)
        print(f"# of epochs = {epoc}\t train_acc = {acc:.4f}")

# of epochs = 5	 train_acc = 0.8885
# of epochs = 10	 train_acc = 0.8977
# of epochs = 15	 train_acc = 0.9130
# of epochs = 20	 train_acc = 0.9232
# of epochs = 25	 train_acc = 0.9318
# of epochs = 30	 train_acc = 0.9308


In [266]:
net.eval()
s = 0 
for Xm,ym in dl_test:
    Xm = Xm.to("cuda:0")
    ym = ym.to("cuda:0")        
    s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
acc = s / len(XX)
print(f"test_acc = {acc:.4f}")

test_acc = 0.8934


In [267]:
sum([torch.tensor(para.shape).prod().item() for para in net.parameters()])

3256330

`-`  아주 깊게.. 

In [273]:
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,256),
    torch.nn.ReLU(),
    torch.nn.Linear(256,256),
    torch.nn.ReLU(),
    torch.nn.Linear(256,256),
    torch.nn.Dropout(0.5),    
    torch.nn.ReLU(),
    torch.nn.Linear(256,256),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(256,10),
).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,31):
    #---에폭시작---#
    net.train()
    for Xm,ym in dl_train:
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        # Step1
        netout = net(Xm)
        # Step2
        loss = loss_fn(netout,ym) 
        # Step3
        loss.backward()
        # Step4 
        optimizr.step()
        optimizr.zero_grad()
    #---에폭끝---#
    if epoc % 5 ==0:
        net.eval()
        s = 0 
        for Xm,ym in dl_train:
            Xm = Xm.to("cuda:0")
            ym = ym.to("cuda:0")        
            s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
        acc = s / len(X)
        print(f"# of epochs = {epoc}\t train_acc = {acc:.4f}")

# of epochs = 5	 train_acc = 0.8903
# of epochs = 10	 train_acc = 0.9157
# of epochs = 15	 train_acc = 0.9281
# of epochs = 20	 train_acc = 0.9333
# of epochs = 25	 train_acc = 0.9424
# of epochs = 30	 train_acc = 0.9524


In [274]:
net.eval()
s = 0 
for Xm,ym in dl_test:
    Xm = Xm.to("cuda:0")
    ym = ym.to("cuda:0")        
    s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
acc = s / len(XX)
print(f"test_acc = {acc:.4f}")

test_acc = 0.8950


In [275]:
sum([torch.tensor(para.shape).prod().item() for para in net.parameters()])

400906

## F. 합성곱신경망

`-` <https://brunch.co.kr/@hvnpoet/109> 

In [277]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=1,out_channels=128,kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Conv2d(in_channels=128,out_channels=128,kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Flatten(),
    torch.nn.Linear(2048,10),
).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
for epoc in range(1,31):
    #---에폭시작---#
    net.train()
    for Xm,ym in dl_train:
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        # Step1
        netout = net(Xm)
        # Step2
        loss = loss_fn(netout,ym) 
        # Step3
        loss.backward()
        # Step4 
        optimizr.step()
        optimizr.zero_grad()
    #---에폭끝---#
    if epoc % 5 ==0:
        net.eval()
        s = 0 
        for Xm,ym in dl_train:
            Xm = Xm.to("cuda:0")
            ym = ym.to("cuda:0")        
            s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
        acc = s / len(X)
        print(f"# of epochs = {epoc}\t train_acc = {acc:.4f}")

# of epochs = 5	 train_acc = 0.9204
# of epochs = 10	 train_acc = 0.9504
# of epochs = 15	 train_acc = 0.9655
# of epochs = 20	 train_acc = 0.9789
# of epochs = 25	 train_acc = 0.9906
# of epochs = 30	 train_acc = 0.9934


In [278]:
net.eval()
s = 0 
for Xm,ym in dl_test:
    Xm = Xm.to("cuda:0")
    ym = ym.to("cuda:0")        
    s = s+(net(Xm).data.argmax(axis=1) == ym.argmax(axis=1)).float().sum().item()
acc = s / len(XX)
print(f"test_acc = {acc:.4f}")

test_acc = 0.9157


In [279]:
sum([torch.tensor(para.shape).prod().item() for para in net.parameters()])

433546